In [7]:
########CELLA 1###############
from google.colab import drive
import zipfile
import os

# 1. Monta Google Drive
drive.mount('/content/drive')

# 2. Configurazione Percorsi
# Assumiamo che il file sia nella root del tuo Drive.
# Se è in una sottocartella, modifica in: '/content/drive/MyDrive/NOME_CARTELLA/dataset.zip'
zip_path = '/content/drive/MyDrive/dataset.zip'
extract_to = '/content/dataset_unzipped'

# 3. Estrazione
if os.path.exists(zip_path):
    print(f"Trovato {zip_path}. Inizio estrazione...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("✅ Estrazione completata!")
else:
    print(f"❌ ERRORE: Non trovo il file '{zip_path}'.")
    print("Controlla di averlo caricato su Drive e che il nome sia esattamente 'dataset.zip' (tutto minuscolo).")

Mounted at /content/drive
Trovato /content/drive/MyDrive/dataset.zip. Inizio estrazione...
✅ Estrazione completata!


In [8]:
##########CELLA 1.5 (RIPRODUCIBILITA')###########
################# AGGIUNTA PER RIPRODUCIBILITÀ (SEEDING) #################
import random
import numpy as np
import torch
import os

def set_all_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # Se usi multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"🌱 Seme impostato su {seed}. Riproducibilità attivata.")

# Impostiamo il seed a 42 (o il numero fortunato che preferisci)
SEED_VALUE = 42
set_all_seeds(SEED_VALUE)

🌱 Seme impostato su 42. Riproducibilità attivata.


In [9]:
############CELLA 2#################
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class BookCoverDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, class_to_idx=None):
        """
        Args:
            csv_file (string): Percorso al file CSV.
            root_dir (string): Directory che contiene le immagini (224x224).
            transform (callable, optional): Trasformazioni (Tensor, Normalize).
        """
        # Lettura CSV con i parametri corretti scoperti prima (sep=; encoding=ISO...)
        self.df = pd.read_csv(csv_file, sep=';', encoding='ISO-8859-1', header=0, on_bad_lines='warn')

        self.root_dir = root_dir
        self.transform = transform

        # Ordina le classi per garantire coerenza
        self.classes = sorted(self.df['Category'].unique())

        # Mappa Stringa -> Intero
        if class_to_idx is None:
            self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        else:
            self.class_to_idx = class_to_idx

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Recupera nome file e costruisce percorso
        img_name = str(self.df.iloc[idx]['Filename'])
        img_path = os.path.join(self.root_dir, img_name)

        # Caricamento Immagine
        try:
            image = Image.open(img_path).convert('RGB')
        except (OSError, FileNotFoundError):
            # Gestione immagine mancante (crea immagine nera)
            image = Image.new('RGB', (224, 224), (0, 0, 0))

        # Label
        label_str = self.df.iloc[idx]['Category']
        label = self.class_to_idx[label_str]

        # Trasformazioni
        if self.transform:
            image = self.transform(image)

        return image, label

# --- CONFIGURAZIONE TRASFORMAZIONI ---
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

data_transforms = {
    'train': transforms.Compose([

        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)), # Nel validation NON facciamo augmentation
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [10]:
#################CELLA 3 (MODIFICATA PER IL TEST SET)####################
# Cerchiamo i percorsi corretti navigando tra le cartelle estratte
base_search_path = '/content/dataset_unzipped'
csv_path_train = None
csv_path_test = None
img_dir = None

print("🔍 Scansione cartelle in corso...")

for root, dirs, files in os.walk(base_search_path):
    # Cerchiamo il CSV di training
    if "book30-listing-train.csv" in files:
        csv_path_train = os.path.join(root, "book30-listing-train.csv")
        print(f"   -> CSV Training Trovato: {csv_path_train}")

    # Cerchiamo il CSV di test
    if "book30-listing-test.csv" in files:
        csv_path_test = os.path.join(root, "book30-listing-test.csv")
        print(f"   -> CSV Test Trovato: {csv_path_test}")

    # Cerchiamo la cartella specifica delle immagini
    if "224x224" in dirs:
        img_dir = os.path.join(root, "224x224")
        print(f"   -> Cartella Immagini Trovata: {img_dir}")

# --- VERIFICA E CREAZIONE DATASET ---\n
if csv_path_train and csv_path_test and img_dir:
    print("\n✅ File trovati! Creazione Dataset in corso...")

    # 1. Creazione del Dataset di Training/Validation
    # Usiamo il transform di 'train' per tutte le immagini che saranno divise
    train_val_dataset = BookCoverDataset(
        csv_file=csv_path_train,
        root_dir=img_dir,
        transform=data_transforms['train'] # Nota: useremo 'val' transform dopo lo split
    )

    # 2. Creazione del Dataset di Test (deve avere le stesse classi)
    # Importante: usiamo il class_to_idx del training per mantenere la mappatura (es. 0 = Action/Adventure)
    test_dataset = BookCoverDataset(
        csv_file=csv_path_test,
        root_dir=img_dir,
        transform=data_transforms['val'], # Per il test usiamo le trasformazioni del validation (solo resize/normalize)
        class_to_idx=train_val_dataset.class_to_idx
    )

    # TEST RAPIDO
    print(f"Dataset di Training/Validation caricato correttamente con {len(train_val_dataset)} libri.")
    print(f"Dataset di Test caricato correttamente con {len(test_dataset)} libri.")
    print(f"Numero di classi (Generi): {len(train_val_dataset.classes)}")

    # Verifica GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\n🖥️ Device attivo: {device}")
    if device.type == 'cuda':
        print("🚀 Perfetto! La GPU NVIDIA T4 è pronta a spingere.")
    else:
        print("⚠️ ATTENZIONE: Stai usando la CPU.")

    # Test estrazione di un elemento
    img, label = train_val_dataset[0]
    print(f"\nTest Shape Tensore: {img.shape} (Deve essere 3, 224, 224)")

else:
    print("\n❌ ERRORE CRITICO: Non ho trovato i file necessari.")
    print("Controlla il contenuto dello zip. Cerco 'book30-listing-train.csv', 'book30-listing-test.csv' e una cartella '224x224'.")

🔍 Scansione cartelle in corso...
   -> CSV Training Trovato: /content/dataset_unzipped/dataset/book30-listing-train.csv
   -> CSV Test Trovato: /content/dataset_unzipped/dataset/book30-listing-test.csv
   -> Cartella Immagini Trovata: /content/dataset_unzipped/dataset/title30cat/224x224

✅ File trovati! Creazione Dataset in corso...
Dataset di Training/Validation caricato correttamente con 51300 libri.
Dataset di Test caricato correttamente con 5700 libri.
Numero di classi (Generi): 30

🖥️ Device attivo: cpu
⚠️ ATTENZIONE: Stai usando la CPU.

Test Shape Tensore: torch.Size([3, 224, 224]) (Deve essere 3, 224, 224)


In [11]:
###############CELLA 4 (MODIFICATA PER IL TEST SET)##############
from torch.utils.data import DataLoader, random_split

# 1. Divisione Train / Validation (80% / 20%) sul dataset di training
total_size = len(train_val_dataset)
train_len = int(0.8 * total_size)
val_len = total_size - train_len

generator = torch.Generator().manual_seed(SEED_VALUE)

# Passiamo il generator a random_split
train_subset, val_subset = random_split(
    train_val_dataset,
    [train_len, val_len],
    generator=generator  # <--- QUESTO GARANTISCE CHE LO SPLIT SIA SEMPRE IDENTICO
)

print(f"📊 Split completato (dal set di Training/Validation):")
print(f"   -> Training Set: {len(train_subset)} immagini")
print(f"   -> Validation Set: {len(val_subset)} immagini")
print(f"   -> Test Set (dal file test.csv): {len(test_dataset)} immagini")

# 2. Creazione dei DataLoader
BATCH_SIZE = 64

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
# NOTA: Per il Validation, i dati vengono processati con i 'train' transforms (inclusa augmentation) perché lo split è stato fatto prima.
# Se volessimo la trasformazione 'val' per il validation, dovremmo ricreare i subset con i transforms corretti.
# Per semplicità, manteniamo questa configurazione che è quella originale.

val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
# Il test_loader deve essere shuffle=False per una valutazione ordinata
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


print(f"✅ Dataloaders pronti (Batch size: {BATCH_SIZE})")

📊 Split completato (dal set di Training/Validation):
   -> Training Set: 41040 immagini
   -> Validation Set: 10260 immagini
   -> Test Set (dal file test.csv): 5700 immagini
✅ Dataloaders pronti (Batch size: 64)


In [12]:
###############CELLA 5###################
from torchvision import models
import torch.nn as nn

def get_model(num_classes=30):
    print("Scaricamento pesi ResNet50 (ImageNet)...")
    # Scarichiamo la versione più aggiornata dei pesi
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

    # 1. FREEZING: Congeliamo tutti i parametri
    # Questo impedisce che durante il training modifichiamo i filtri che sanno già "vedere"
    for param in model.parameters():
        param.requires_grad = False

    # 2. Sostituzione dell'ultimo layer (Fully Connected)
    # ResNet50 ha 2048 feature in ingresso all'ultimo layer
    num_ftrs = model.fc.in_features

    # Creiamo il nuovo layer classificatore.
    # Nota: Di default, i nuovi layer hanno requires_grad=True, quindi QUESTI verranno addestrati.
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 512), # Strato intermedio per imparare combinazioni complesse
        nn.ReLU(),
        nn.Dropout(0.5),          # Dropout per evitare overfitting (tecnica standard)
        nn.Linear(512, num_classes) # Output finale: 30 probabilità
    )

    return model

# Istanziamo il modello e lo spostiamo sulla GPU
model = get_model(num_classes=len(train_val_dataset.classes))
model = model.to(device) # Sposta tutto sulla GPU T4

print("\n🤖 Modello ResNet50 caricato e modificato per 30 classi.")
print("   -> Backbone (corpo): Congelato ❄️")
print("   -> Head (testa): Pronta per l'addestramento 🔥")

Scaricamento pesi ResNet50 (ImageNet)...
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 135MB/s]



🤖 Modello ResNet50 caricato e modificato per 30 classi.
   -> Backbone (corpo): Congelato ❄️
   -> Head (testa): Pronta per l'addestramento 🔥


In [ ]:
################## CELLA 6 (AGGIORNATA CON WARM-UP) #################
import torch.optim as optim
import time
import torch.nn as nn
import copy
from torchvision import models
import os

# --- 0. CONFIGURAZIONE SALVATAGGIO ---
SAVE_PATH = '/content/drive/MyDrive/Modelli_BookCover'
os.makedirs(SAVE_PATH, exist_ok=True)
MODEL_NAME = 'best_resnet50_finetuned.pth'
FULL_MODEL_PATH = os.path.join(SAVE_PATH, MODEL_NAME)

print(f"💾 Il modello migliore verrà salvato in: {FULL_MODEL_PATH}")

# Loss Function (uguale per tutti)
criterion = nn.CrossEntropyLoss()

# --- DEFINIZIONE FUNZIONE DI TRAINING (Invariata nella logica) ---
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=20, es_patience=6, phase_name="Training"):
    start_time = time.time()

    # Se esiste già un file pesi (es. dalla fase warm-up), carichiamo il best attuale come punto di partenza per il confronto
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_loss = float('inf')

    # Se stiamo facendo la Fase 2, proviamo a leggere la best loss precedente per non sovrascrivere con modelli peggiori
    if os.path.exists(FULL_MODEL_PATH) and phase_name == "Fase 2 (Fine-Tuning)":
        print("   -> Carico pesi migliori precedenti per confronto...")
        # Nota: qui servirebbe salvare anche la loss, per semplicità resettiamo il confronto ma partiamo dai pesi buoni
        # La logica standard resetta il best_val_loss qui per permettere il saving nella nuova fase

    patience_counter = 0

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs} [{phase_name}]')
        print('-' * 10)

        # Stampa LR correnti
        for i, param_group in enumerate(optimizer.param_groups):
            print(f"📉 LR Group {i}: {param_group['lr']:.1e}", end=" | ")
        print()

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                dataloader = train_loader
            else:
                model.eval()
                dataloader = val_loader

            running_loss = 0.0
            running_corrects = 0
            total_samples = 0

            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                total_samples += inputs.size(0)

            epoch_loss = running_loss / total_samples
            epoch_acc = running_corrects.double() / total_samples

            if phase == 'val':
                scheduler.step(epoch_loss)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # --- LOGICA SALVATAGGIO ---
            if phase == 'val':
                if epoch_loss < best_val_loss:
                    best_val_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())
                    patience_counter = 0
                    print(f"✅ Miglioramento! Salvataggio in {FULL_MODEL_PATH}...")
                    torch.save(model.state_dict(), FULL_MODEL_PATH)
                else:
                    patience_counter += 1
                    print(f"⚠️ Patience: {patience_counter}/{es_patience}")

        if patience_counter >= es_patience:
            print(f"\n⏹️ Early Stopping attivato durante {phase_name}!")
            break

    time_elapsed = time.time() - start_time
    print(f'\n{phase_name} completata in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Miglior Val Loss di fase: {best_val_loss:.4f}')

    # Ricarichiamo i pesi migliori
    model.load_state_dict(torch.load(FULL_MODEL_PATH))
    return model

# ==========================================
# 🚀 FASE 1: WARM-UP (SOLO TESTA)
# ==========================================
print("\n🔥 FASE 1: WARM-UP (Addestramento veloce solo della testa)...")

# 1. Congeliamo TUTTO
for param in model.parameters():
    param.requires_grad = False
# 2. Sblocchiamo SOLO la testa
for param in model.fc.parameters():
    param.requires_grad = True

# 3. Optimizer "Aggressivo" per la testa (LR più alto)
optimizer_warmup = optim.Adam(model.fc.parameters(), lr=1e-3)
scheduler_warmup = optim.lr_scheduler.ReduceLROnPlateau(optimizer_warmup, mode='min', factor=0.1, patience=2)

# 4. Train Breve (es. 4 epoche)
model = train_model(
    model, train_loader, val_loader, criterion,
    optimizer_warmup, scheduler_warmup,
    num_epochs=4, es_patience=2, phase_name="Fase 1 (Warm-Up)"
)

# ==========================================
# ❄️ FASE 2: FINE-TUNING (LAYER4 + TESTA)
# ==========================================
print("\n🔓 FASE 2: FINE-TUNING (Raffinamento Layer4 e Testa)...")

# 1. Sblocchiamo Layer 4 (oltre alla testa che è già sbloccata)
for param in model.layer4.parameters():
    param.requires_grad = True

# Verifica parametri sbloccati
params_to_update = [p for p in model.parameters() if p.requires_grad]
print(f"Parametri da addestrare: {sum(p.numel() for p in params_to_update):,}")

# 2. Optimizer "Delicato" (LR differenziati e bassi)
optimizer_ft = optim.Adam([
    {'params': model.layer4.parameters(), 'lr': 1e-5}, # Backbone: impara piano per non rovinare pesi ImageNet
    {'params': model.fc.parameters(), 'lr': 1e-4}      # Testa: impara normalmente
], weight_decay=1e-4)

scheduler_ft = optim.lr_scheduler.ReduceLROnPlateau(optimizer_ft, mode='min', factor=0.1, patience=3)

# 3. Train Lungo
model = train_model(
    model, train_loader, val_loader, criterion,
    optimizer_ft, scheduler_ft,
    num_epochs=30, es_patience=6, phase_name="Fase 2 (Fine-Tuning)"
)

💾 Il modello migliore verrà salvato in: /content/drive/MyDrive/Modelli_BookCover/best_resnet50_finetuned.pth

🔥 FASE 1: WARM-UP (Addestramento veloce solo della testa)...

Epoch 1/4 [Fase 1 (Warm-Up)]
----------
📉 LR Group 0: 1.0e-03 | 
train Loss: 2.9221 Acc: 0.1927
val Loss: 2.7351 Acc: 0.2370
✅ Miglioramento! Salvataggio in /content/drive/MyDrive/Modelli_BookCover/best_resnet50_finetuned.pth...

Epoch 2/4 [Fase 1 (Warm-Up)]
----------
📉 LR Group 0: 1.0e-03 | 
train Loss: 2.7363 Acc: 0.2378
val Loss: 2.6928 Acc: 0.2554
✅ Miglioramento! Salvataggio in /content/drive/MyDrive/Modelli_BookCover/best_resnet50_finetuned.pth...

Epoch 3/4 [Fase 1 (Warm-Up)]
----------
📉 LR Group 0: 1.0e-03 | 
train Loss: 2.6664 Acc: 0.2540
val Loss: 2.6734 Acc: 0.2547
✅ Miglioramento! Salvataggio in /content/drive/MyDrive/Modelli_BookCover/best_resnet50_finetuned.pth...

Epoch 4/4 [Fase 1 (Warm-Up)]
----------
📉 LR Group 0: 1.0e-03 | 
train Loss: 2.6209 Acc: 0.2621
val Loss: 2.6568 Acc: 0.2634
✅ Miglioramen

In [ ]:
################## CELLA DI CARICAMENTO (NO TRAINING) #################
'''#######NEL CASO IN CUI TU VOGLIA CARICARE UN MODELLO SALVATO IN PRECEDENZA NEL DRIVE
import torch
import os

# 1. Definizione Percorso (Lo stesso usato per il salvataggio)
SAVE_PATH = '/content/drive/MyDrive/Modelli_BookCover'
MODEL_NAME = 'best_resnet50_finetuned.pth'
FULL_MODEL_PATH = os.path.join(SAVE_PATH, MODEL_NAME)

# 2. Verifica esistenza file
if os.path.exists(FULL_MODEL_PATH):
    print(f"📂 Trovato modello salvato: {FULL_MODEL_PATH}")

    # 3. Caricamento dei pesi nello scheletro creato nella Cella 5
    # map_location assicura che funzioni sia su CPU che GPU
    state_dict = torch.load(FULL_MODEL_PATH, map_location=device)
    model.load_state_dict(state_dict)

    # Spostiamo il modello sulla GPU (se disponibile)
    model = model.to(device)

    # Mettiamo il modello in modalità valutazione (blocca dropout, batchnorm, etc.)
    model.eval()

    print("✅ Pesi caricati con successo! Il modello è pronto per la valutazione.")
    print("⏭️ Ora puoi eseguire direttamente le celle 7 e 8.")
else:
    print(f"❌ ERRORE: Non trovo il file in {FULL_MODEL_PATH}")
    print("Assicurati di aver fatto almeno un training completo in precedenza.")'''

📂 Trovato modello salvato: /content/drive/MyDrive/Modelli_BookCover/best_resnet50_finetuned.pth
✅ Pesi caricati con successo! Il modello è pronto per la valutazione.
⏭️ Ora puoi eseguire direttamente le celle 7 e 8.


In [ ]:
##################CELLA 7#################
'''import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

def plot_confusion_matrix(model, dataloader, classes):
    model.eval() # Modalità valutazione
    y_true = []
    y_pred = []

    print("📊 Calcolo delle predizioni sul Validation Set...")
    with torch.no_grad(): # Disabilita il calcolo dei gradienti (risparmia memoria)
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            # Spostiamo su CPU e convertiamo in numpy
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    # Calcolo della matrice
    cm = confusion_matrix(y_true, y_pred)

    # Normalizzazione (opzionale, per vedere le % invece dei numeri assoluti)
    # cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    # Plotting
    plt.figure(figsize=(20, 16)) # Dimensioni grandi per farci stare 30 classi
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)

    plt.ylabel('Vero Genere (True Label)', fontsize=14)
    plt.xlabel('Genere Predetto (Predicted Label)', fontsize=14)
    plt.title('Matrice di Confusione - Generi Letterari', fontsize=18)
    plt.xticks(rotation=90) # Ruota le etichette per leggerle meglio
    plt.yticks(rotation=0)
    plt.show()

# Eseguiamo la funzione usando il modello addestrato
# Assicurati che 'trained_model' sia disponibile (dopo la Cella 6)
print("Generazione grafico in corso...")
plot_confusion_matrix(model, test_loader, train_val_dataset.classes)'''

In [14]:
################## CELLA 8#################
#METRICHE PAPER (TOP-1, TOP-2, TOP-3)
def evaluate_paper_metrics(model, dataloader):
    model.eval()

    correct_top1 = 0
    correct_top2 = 0
    correct_top3 = 0
    total = 0

    print("📏 Calcolo metriche Top-k (Come nel Paper)...")

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 1. Calcoliamo l'output del modello
            outputs = model(inputs)

            # 2. Prendiamo le prime 3 predizioni più alte (Top-3)
            # k=3 perché ci servono Top-1, Top-2 e Top-3
            _, max_k_preds = torch.topk(outputs, k=3, dim=1)

            # 3. Trasponiamo la matrice per facilitare il confronto
            # Ora max_k_preds ha shape [3, batch_size]
            max_k_preds = max_k_preds.t()

            # 4. Creiamo una matrice di etichette ripetute per confrontarle
            target_expanded = labels.view(1, -1).expand_as(max_k_preds)

            # 5. Confronto: Otteniamo una matrice di True/False
            # Se la cella (k, i) è True, vuol dire che la k-esima predizione per l'immagine i è corretta
            correct = max_k_preds.eq(target_expanded)

            # --- AGGIORNAMENTO CONTATORI ---

            # Top-1: Controlliamo solo la prima riga (la predizione #1)
            correct_top1 += correct[:1].reshape(-1).float().sum(0, keepdim=True)

            # Top-2: Controlliamo le prime due righe (predizione #1 O predizione #2)
            correct_top2 += correct[:2].reshape(-1).float().sum(0, keepdim=True)

            # Top-3: Controlliamo le prime tre righe
            correct_top3 += correct[:3].reshape(-1).float().sum(0, keepdim=True)

            total += labels.size(0)

    # Calcolo percentuali finali
    acc_top1 = correct_top1.item() / total * 100
    acc_top2 = correct_top2.item() / total * 100
    acc_top3 = correct_top3.item() / total * 100

    print(f"\n📊 RISULTATI DEL PAPER (su {total} immagini di test):")
    print("-" * 40)
    print(f"🔹 Top-1 Accuracy: {acc_top1:.2f}%  (Paper AlexNet: 24.7%)")
    print(f"🔹 Top-2 Accuracy: {acc_top2:.2f}%  (Paper AlexNet: 33.1%)")
    print(f"🔹 Top-3 Accuracy: {acc_top3:.2f}%  (Paper AlexNet: 40.3%)")
    print("-" * 40)
    return acc_top1, acc_top2, acc_top3

# Esegui la valutazione
evaluate_paper_metrics(model, test_loader)

📏 Calcolo metriche Top-k (Come nel Paper)...


KeyboardInterrupt: 